In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [4]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available: 0


In [5]:
datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2,
    rotation_range=20,  # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,  # Randomly apply shearing transformation
    zoom_range=0.2,  # Randomly zoom inside images
    horizontal_flip=True  # Randomly flip images horizontally
)

In [6]:
train_generator = datagen.flow_from_directory(
    directory='Animal Images',  
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training',
    shuffle=True,
    seed=42
)

test_generator = datagen.flow_from_directory(
    directory='Animal Images',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=True,
    seed=42
)

Found 24050 images belonging to 2 classes.
Found 6011 images belonging to 2 classes.


In [7]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [8]:
model.compile(optimizer = Adam(learning_rate = 1e-3), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [9]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
history = model.fit(train_generator, epochs=15, validation_data=test_generator, callbacks=[early_stopping])

Epoch 1/15


752/752 [==============================] - 463s 613ms/step - loss: 0.6939 - accuracy: 0.5154 - val_loss: 0.6806 - val_accuracy: 0.6079
Epoch 2/15
752/752 [==============================] - 462s 614ms/step - loss: 0.6762 - accuracy: 0.5763 - val_loss: 0.6573 - val_accuracy: 0.6024
Epoch 3/15
752/752 [==============================] - 459s 610ms/step - loss: 0.6455 - accuracy: 0.6307 - val_loss: 0.6386 - val_accuracy: 0.6277
Epoch 4/15
752/752 [==============================] - 468s 622ms/step - loss: 0.6306 - accuracy: 0.6517 - val_loss: 0.6243 - val_accuracy: 0.6661
Epoch 5/15
260/752 [=========>....................] - ETA: 4:30 - loss: 0.6179 - accuracy: 0.6600

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()